# 消息格式处理

Meesages 类通过对字符串、模板、元组等类型的兼容，以及默认的 role 生成规则，来快速构造消息列表。

大模型所需要的消息列表格式，通常是这样：
```python
[
    {'role': 'system', 'content': '你是一个小说家。'},
    {'role': 'user', 'content': '帮我创作吧'},
    {'role': 'assistant', 'content': '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'}
]
 ```

这有些啰嗦，可以这样来简化：
```python
Messages([
    '你是一个小说家。',
    '帮我创作吧',
    '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'
])
 ```

In [1]:
from illufly.types import Message, Messages, Template

## role 生成规则

### 规则1： 不指定时，首条是 system，system 后接 user，user 与 assistant 交替

In [2]:
Messages(["hi"]).to_list()

[{'role': 'system', 'content': 'hi'}]

In [3]:
Messages(["hi", "我很开心", "我也很开心"]).to_list()

[{'role': 'system', 'content': 'hi'},
 {'role': 'user', 'content': '我很开心'},
 {'role': 'assistant', 'content': '我也很开心'}]

In [4]:
Messages([
    Template(text
    "帮我创作吧",
    "从前有一个人很坏，他坏死了。\n额，我是他说真的死了。"
]).to_list()

[{'role': 'system', 'content': '你是一个小说家。'},
 {'role': 'user', 'content': '帮我创作吧'},
 {'role': 'assistant', 'content': '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'}]

In [5]:
Messages([
    ("system", Template(text
    "我这个小说很有趣哦，我结束了",
]).to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '我这个小说很有趣哦，我结束了'}]

In [6]:
Messages([
    ("user", Template(text
    {"role": "assistant", "content": "我开始了"},
]).to_list()

[{'role': 'user', 'content': '你是一个小说家，帮我创作一个说。'},
 {'role': 'assistant', 'content': '我开始了'}]

### 规则2：列表中允许出现字符串、Turple、Template、Message、Dict等类型

In [7]:
Messages([
    Template(text="你是一个小说家，帮我创作吧。"),
    ("user", "不错"),
    {"role": "assistant", "content": "请你开始"},
    "我这个小说很有趣哦，我结束了",
]).raw_messages

[<Template template_vars=set() template_text='你是一个小说家，帮我创作吧。'>,
 ('user', '不错'),
 {'role': 'assistant', 'content': '请你开始'},
 '我这个小说很有趣哦，我结束了']

In [8]:
Messages([
    Template(text
    ("user", "不错"),
    {"role": "assistant", "content": "请你开始"},
    "我这个小说很有趣哦，我结束了",
]).to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '不错'},
 {'role': 'assistant', 'content': '请你开始'},
 {'role': 'user', 'content': '我这个小说很有趣哦，我结束了'}]

### 规则3：允许用 ai 替代 assistant 作为 role 来声明，但会被替换为 assistant

In [9]:
Messages([
    ("user", Template(text
    ("ai", "我开始了"),
    {"role": "user", "content": "我开始了"},
    {"role": "ai", "content": "我开始了"},
]).to_list()

[{'role': 'user', 'content': '你是一个小说家，帮我创作一个说。'},
 {'role': 'assistant', 'content': '我开始了'},
 {'role': 'user', 'content': '我开始了'},
 {'role': 'assistant', 'content': '我开始了'}]

## 工具回调消息

当消息列表中包含工具回调消息时，要求兼容：

In [1]:
from illufly.types import Messages

messages = Messages([{'role': 'user', 'content': '几点了'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'role': 'tool',
  'name': 'get_current_time',
  'content': '当前时间：2024-09-28 14:33:56。'},
 {'role': 'assistant', 'content': '现在的时间是2024年9月28日 14点33分56秒。'},
 {'role': 'user', 'content': '现在几点了？'}])

messages.to_list()

[{'role': 'user', 'content': '几点了'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'role': 'tool',
  'content': '当前时间：2024-09-28 14:33:56。',
  'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'name': 'get_current_time'},
 {'role': 'assistant', 'content': '现在的时间是2024年9月28日 14点33分56秒。'},
 {'role': 'user', 'content': '现在几点了？'}]

In [2]:
messages.to_list(style="openai_vl")

[{'role': 'user', 'content': [{'type': 'text', 'text': '几点了'}]},
 {'role': 'assistant',
  'content': [{'type': 'text', 'text': ''}],
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'role': 'tool',
  'content': [{'type': 'text', 'text': '当前时间：2024-09-28 14:33:56。'}],
  'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'name': 'get_current_time'},
 {'role': 'assistant',
  'content': [{'type': 'text', 'text': '现在的时间是2024年9月28日 14点33分56秒。'}]},
 {'role': 'user', 'content': [{'type': 'text', 'text': '现在几点了？'}]}]

In [33]:
messages.to_list(style="qwen_vl")

[{'role': 'user', 'content': [{'text': '几点了'}]},
 {'role': 'assistant', 'content': [{'text': ''}]},
 {'role': 'tool', 'content': [{'text': '当前时间：2024-09-28 14:33:56。'}]},
 {'role': 'assistant', 'content': [{'text': '现在的时间是2024年9月28日 14点33分56秒。'}]},
 {'role': 'user', 'content': [{'text': '现在几点了？'}]}]

In [36]:
messages.raw_messages

[{'role': 'user', 'content': '几点了'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_8b4dfce376834e2295a4ea',
    'type': 'function',
    'function': {'name': 'get_current_time', 'arguments': '{}'}}]},
 {'tool_call_id': 'call_8b4dfce376834e2295a4ea',
  'role': 'tool',
  'name': 'get_current_time',
  'content': '当前时间：2024-09-28 14:33:56。'},
 {'role': 'assistant', 'content': '现在的时间是2024年9月28日 14点33分56秒。'},
 {'role': 'user', 'content': '现在几点了？'}]

## 多模态消息

### 多模态消息的定义兼容原有格式

In [10]:
Messages([
    ("user", Template(text
    ("ai", "我开始了"),
    {"role": "user", "content": "我开始了"},
    {"role": "ai", "content": "我开始了"},
]).to_list(input_vars={}, style="qwen_vl")

[{'role': 'user', 'content': [{'text': '你是一个小说家，帮我创作一个说。'}]},
 {'role': 'assistant', 'content': [{'text': '我开始了'}]},
 {'role': 'user', 'content': [{'text': '我开始了'}]},
 {'role': 'assistant', 'content': [{'text': '我开始了'}]}]

In [11]:
Template(text()

'你是一个小说家，帮我创作一个说。'

### OpenAI风格的多模态消息格式

In [12]:
openai_msg = Messages(
    [
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": "https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": "https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png"
              }
            },
            {
              "type": "text",
              "text": "这些是什么"
            }
          ]
        }
    ],
    style="openai_vl"
)

In [13]:
openai_msg.to_list()

[{'role': 'user',
  'content': [{'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'}},
   {'type': 'text', 'text': '这些是什么'}]}]

In [14]:
openai_msg.to_list(style="qwen_vl")

[{'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]}]

In [15]:
openai_msg.to_list(style="text")

[{'role': 'user', 'content': '这些是什么'}]

### 通义千问的多模态消息格式

In [16]:
qwen_msg = Messages([
    ("user", [
        {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
        {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
        {'text': '这些是什么'}
    ])
], style="qwen_vl")

In [17]:
qwen_msg.to_list()

[{'role': 'user',
  'content': [{'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'},
   {'image': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'},
   {'text': '这些是什么'}]}]

In [18]:
qwen_msg.to_list(style="openai_vl")

[{'role': 'user',
  'content': [{'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://dashscope.oss-cn-beijing.aliyuncs.com/images/tiger.png'}},
   {'type': 'text', 'text': '这些是什么'}]}]

In [19]:
qwen_msg.to_list(style="text")

[{'role': 'user', 'content': '这些是什么'}]

## 提示语模板动态填充

使用 Template 可以根据映射规则，动态提取数据，完成模板组装。

In [20]:
from illufly.types import Template, Messages

t = Template("IDEA")
t

<Template template_vars={'task'} template_text='你是强大的写作助手。...'>

### 模板输入变量映射

In [21]:
# 默认映射
Template("IDEA").imported_vars

{}

In [22]:
# 指定映射
Template("IDEA", binding_map={"task": "question"}).imported_vars

{}

### 填充模板变量

In [23]:
# 格式化
Template("IDEA").format({"task": "请帮我写一首歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请帮我写一首歌\n'

In [24]:
# 映射变量缺失
Template("IDEA")(verbose=True)

  0s [INFO] 你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出

**你的任务是:**




'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n\n'

In [25]:
# 改变映射规则
Template(
    "IDEA",
    input_mapping={"task": "question"}
).format({"question": "请帮我写一首歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n\n'

In [26]:
# 映射到深层嵌套的字典变量
Template(
    "IDEA",
    input_mapping={"task": "state.my.question"}
).format({"state": {"my": {"question": "请帮我写一首歌"}}})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n\n'

In [27]:
messages = Messages(["hi", '我想回家', "有什么可以帮您?"])
messages.input_vars

[]

### 使用提示语模板构建消息

In [28]:
from illufly.types import Messages, Message, Template

Message("user", Template(
    "IDEA",
    input_mapping={"task": "question"}
))

Message(role=user, content=你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出

**你的任务是:**

)